# Generate Data

1. Copy stereo images into single folder
2. Rotate vertical images, so all images are aligned horizontally
3. From 1280 x 720px images, create 5 x 3 single images (l + r image) through windowing + slicing
4. Create anaglyph + inverse anaglyph from stereo images (with dame ID, but suffix)
5. Split data 70/30 into training and test data

## Imports

In [1]:
import numpy as np
import cv2 as cv2
import os
from sklearn.model_selection import train_test_split
import shutil
from tqdm import tqdm

In [2]:
source_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\holopix50k'
# source_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\test_data'
anaglyph_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\anaglyph'
inverse_anaglyph_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\inverse_anaglyph'
stereo_slice_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\stereo'
stereo_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\stereo_full'

## 2. Rotate images

In [3]:
def rotate_vertical_image(img):
    if img.shape[0] > img.shape[1]:
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    return img


## 3. Slice images

In [4]:
def create_image_chunks(image, num_rows = 3, num_cols = 5, chunk_height = 256, chunk_width = 256, overlap = 24):
    chunks = []
    for row in range(num_rows):
        for col in range(num_cols):
            x_start = col * (chunk_width - overlap)
            y_start = row * (chunk_height - overlap)
            chunks.append(image[y_start:y_start + chunk_height, x_start:x_start + chunk_width])
    return chunks

## 4. Create Anaglyph

In [5]:
def split_image(image):
    # Get the image shape
    height, width, _ = image.shape

    # Calculate the middle of the image
    middle = width // 2

    # Split the image into left and right halves
    left_img = image[:, :middle]
    right_img = image[:, middle:]

    return left_img, right_img

In [6]:
def merge_channels(img_left, img_right):
    # Split the images into B,G,R channels
    B_left, G_left, R_left = cv2.split(img_left)
    B_right, G_right, R_right = cv2.split(img_right)

    # Create a new image with the red channel from 'img_left' and the blue + green channel from 'img_right'
    merged_anaglyph = cv2.merge((B_right, G_right, R_left))
    merged_inverse_anaglyph = cv2.merge((B_left, G_left, R_right))

    return merged_anaglyph, merged_inverse_anaglyph

## 5. Split data

In [7]:
# X = ...  # Your data (e.g., anaglyphs)
# y = ...  # Labels (e.g., image IDs)
# 
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Run for all

In [8]:
def process_all_images(source_folder, pbar):
     # Initialize a counter
    counter = 0
    
    # Walk through the source directory
    for file in os.listdir(source_folder):
        # Check if the file is a .png
        if file.endswith('.jpg'):
            # Create the full file paths
            src_file_path = os.path.join(source_folder, file)

            # Get image name + ext
            base_name, ext = os.path.splitext(file)
            
            # Read the image
            img = cv2.imread(src_file_path)

            # Rotate the image if vertical
            img = rotate_vertical_image(img)
            cv2.imwrite(os.path.join(stereo_folder, base_name + ext), img)
            
            # Split image
            img_left, img_right = split_image(img)
            
            # Create image slices
            img_chunks_left = create_image_chunks(img_left)
            img_chunks_right = create_image_chunks(img_right)
            
            # Create anaglyphs
            for i in range(len(img_chunks_left)):
                img_anaglyph, img_inverse_anaglyph = merge_channels(img_chunks_left[i], img_chunks_right[i])
                cv2.imwrite(os.path.join(stereo_slice_folder, base_name + f'_{i}_left' + ext), img_chunks_left[i])
                cv2.imwrite(os.path.join(stereo_slice_folder, base_name + f'_{i}_right' + ext), img_chunks_right[i])
                cv2.imwrite(os.path.join(anaglyph_folder, base_name + f'_{i}_anaglyph' + ext), img_anaglyph)
                cv2.imwrite(os.path.join(inverse_anaglyph_folder, base_name + f'_{i}_inverseanaglyph' + ext), img_inverse_anaglyph)

            # Increment the counter
            counter += 1
            # Update the progress bar
            pbar.update()

In [9]:
 # Get the total number of .jpg files in the source directory
total_files = len([file for file in os.listdir(source_folder) if file.endswith('.jpg')])



# Copy the images with a progress bar
with tqdm(total=total_files, desc="Processing images", unit="file") as pbar:
    process_all_images(source_folder, pbar)

Processing images:   0%|          | 8/49124 [00:01<1:49:00,  7.51file/s]


ValueError: not enough values to unpack (expected 3, got 0)